In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("WindowFunctionsExample") \
    .getOrCreate()

25/09/17 05:16:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = spark.read.options(header='True',inferSchema='True').csv("/home/iceberg/notebooks/data/data.csv")
df.show()

+---+--------+------+----------+----------+
| id|employee|amount|department|      date|
+---+--------+------+----------+----------+
|  1|   Alice|   100|         A|2025-09-10|
|  2|     Bob|   150|         A|2025-09-11|
|  3| Charlie|   200|         B|2025-09-12|
|  4|   Alice|   250|         A|2025-09-13|
|  5|     Bob|   300|         A|2025-09-14|
|  6| Charlie|   150|         B|2025-09-15|
|  7|   Alice|   300|         A|2025-09-16|
|  8|     Bob|   200|         A|2025-09-17|
|  9| Charlie|   250|         B|2025-09-18|
+---+--------+------+----------+----------+



In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number,rank

In [18]:
windowSpec = Window.partitionBy("employee").orderBy("date")

In [19]:
windowSpecAgg = Window.partitionBy("employee")

In [15]:
df.withColumn("row number",row_number().over(windowSpec)).show()

+---+--------+------+----------+----------+----------+
| id|employee|amount|department|      date|row number|
+---+--------+------+----------+----------+----------+
|  1|   Alice|   100|         A|2025-09-10|         1|
|  4|   Alice|   250|         A|2025-09-13|         2|
|  7|   Alice|   300|         A|2025-09-16|         3|
|  2|     Bob|   150|         A|2025-09-11|         1|
|  5|     Bob|   300|         A|2025-09-14|         2|
|  8|     Bob|   200|         A|2025-09-17|         3|
|  3| Charlie|   200|         B|2025-09-12|         1|
|  6| Charlie|   150|         B|2025-09-15|         2|
|  9| Charlie|   250|         B|2025-09-18|         3|
+---+--------+------+----------+----------+----------+



In [20]:
df.withColumn("rank",rank().over(windowSpec)).show()

+---+--------+------+----------+----------+----+
| id|employee|amount|department|      date|rank|
+---+--------+------+----------+----------+----+
|  1|   Alice|   100|         A|2025-09-10|   1|
|  4|   Alice|   250|         A|2025-09-13|   2|
|  7|   Alice|   300|         A|2025-09-16|   3|
|  2|     Bob|   150|         A|2025-09-11|   1|
|  5|     Bob|   300|         A|2025-09-14|   2|
|  8|     Bob|   200|         A|2025-09-17|   3|
|  3| Charlie|   200|         B|2025-09-12|   1|
|  6| Charlie|   150|         B|2025-09-15|   2|
|  9| Charlie|   250|         B|2025-09-18|   3|
+---+--------+------+----------+----------+----+



In [21]:
from pyspark.sql.functions import dense_rank
df.withColumn("Dense Rank",dense_rank().over(windowSpec)).show()

+---+--------+------+----------+----------+----------+
| id|employee|amount|department|      date|Dense Rank|
+---+--------+------+----------+----------+----------+
|  1|   Alice|   100|         A|2025-09-10|         1|
|  4|   Alice|   250|         A|2025-09-13|         2|
|  7|   Alice|   300|         A|2025-09-16|         3|
|  2|     Bob|   150|         A|2025-09-11|         1|
|  5|     Bob|   300|         A|2025-09-14|         2|
|  8|     Bob|   200|         A|2025-09-17|         3|
|  3| Charlie|   200|         B|2025-09-12|         1|
|  6| Charlie|   150|         B|2025-09-15|         2|
|  9| Charlie|   250|         B|2025-09-18|         3|
+---+--------+------+----------+----------+----------+



In [22]:
from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(2).over(windowSpec)).show()

+---+--------+------+----------+----------+-----+
| id|employee|amount|department|      date|ntile|
+---+--------+------+----------+----------+-----+
|  1|   Alice|   100|         A|2025-09-10|    1|
|  4|   Alice|   250|         A|2025-09-13|    1|
|  7|   Alice|   300|         A|2025-09-16|    2|
|  2|     Bob|   150|         A|2025-09-11|    1|
|  5|     Bob|   300|         A|2025-09-14|    1|
|  8|     Bob|   200|         A|2025-09-17|    2|
|  3| Charlie|   200|         B|2025-09-12|    1|
|  6| Charlie|   150|         B|2025-09-15|    1|
|  9| Charlie|   250|         B|2025-09-18|    2|
+---+--------+------+----------+----------+-----+



In [23]:
from pyspark.sql.functions import sum
df.withColumn("running_sum",sum("amount").over(windowSpec)).show()

+---+--------+------+----------+----------+-----------+
| id|employee|amount|department|      date|running_sum|
+---+--------+------+----------+----------+-----------+
|  1|   Alice|   100|         A|2025-09-10|        100|
|  4|   Alice|   250|         A|2025-09-13|        350|
|  7|   Alice|   300|         A|2025-09-16|        650|
|  2|     Bob|   150|         A|2025-09-11|        150|
|  5|     Bob|   300|         A|2025-09-14|        450|
|  8|     Bob|   200|         A|2025-09-17|        650|
|  3| Charlie|   200|         B|2025-09-12|        200|
|  6| Charlie|   150|         B|2025-09-15|        350|
|  9| Charlie|   250|         B|2025-09-18|        600|
+---+--------+------+----------+----------+-----------+



In [24]:
from pyspark.sql.functions import avg
df.withColumn("running_avg",avg("amount").over(windowSpec)).show()

+---+--------+------+----------+----------+------------------+
| id|employee|amount|department|      date|       running_avg|
+---+--------+------+----------+----------+------------------+
|  1|   Alice|   100|         A|2025-09-10|             100.0|
|  4|   Alice|   250|         A|2025-09-13|             175.0|
|  7|   Alice|   300|         A|2025-09-16|216.66666666666666|
|  2|     Bob|   150|         A|2025-09-11|             150.0|
|  5|     Bob|   300|         A|2025-09-14|             225.0|
|  8|     Bob|   200|         A|2025-09-17|216.66666666666666|
|  3| Charlie|   200|         B|2025-09-12|             200.0|
|  6| Charlie|   150|         B|2025-09-15|             175.0|
|  9| Charlie|   250|         B|2025-09-18|             200.0|
+---+--------+------+----------+----------+------------------+



In [25]:
from pyspark.sql.functions import min
df.withColumn("min_amount",min("amount").over(windowSpecAgg)).show()

+---+--------+------+----------+----------+----------+
| id|employee|amount|department|      date|min_amount|
+---+--------+------+----------+----------+----------+
|  1|   Alice|   100|         A|2025-09-10|       100|
|  4|   Alice|   250|         A|2025-09-13|       100|
|  7|   Alice|   300|         A|2025-09-16|       100|
|  2|     Bob|   150|         A|2025-09-11|       150|
|  5|     Bob|   300|         A|2025-09-14|       150|
|  8|     Bob|   200|         A|2025-09-17|       150|
|  3| Charlie|   200|         B|2025-09-12|       150|
|  6| Charlie|   150|         B|2025-09-15|       150|
|  9| Charlie|   250|         B|2025-09-18|       150|
+---+--------+------+----------+----------+----------+



In [26]:
from pyspark.sql.functions import max
df.withColumn("max_amount",max("amount").over(windowSpecAgg)).show()

+---+--------+------+----------+----------+----------+
| id|employee|amount|department|      date|max_amount|
+---+--------+------+----------+----------+----------+
|  1|   Alice|   100|         A|2025-09-10|       300|
|  4|   Alice|   250|         A|2025-09-13|       300|
|  7|   Alice|   300|         A|2025-09-16|       300|
|  2|     Bob|   150|         A|2025-09-11|       300|
|  5|     Bob|   300|         A|2025-09-14|       300|
|  8|     Bob|   200|         A|2025-09-17|       300|
|  3| Charlie|   200|         B|2025-09-12|       250|
|  6| Charlie|   150|         B|2025-09-15|       250|
|  9| Charlie|   250|         B|2025-09-18|       250|
+---+--------+------+----------+----------+----------+



In [29]:
from pyspark.sql.functions import lead
df.withColumn("lead_amount",lead("amount", 1).over(windowSpec)).show()

+---+--------+------+----------+----------+-----------+
| id|employee|amount|department|      date|lead_amount|
+---+--------+------+----------+----------+-----------+
|  1|   Alice|   100|         A|2025-09-10|        250|
|  4|   Alice|   250|         A|2025-09-13|        300|
|  7|   Alice|   300|         A|2025-09-16|       NULL|
|  2|     Bob|   150|         A|2025-09-11|        300|
|  5|     Bob|   300|         A|2025-09-14|        200|
|  8|     Bob|   200|         A|2025-09-17|       NULL|
|  3| Charlie|   200|         B|2025-09-12|        150|
|  6| Charlie|   150|         B|2025-09-15|        250|
|  9| Charlie|   250|         B|2025-09-18|       NULL|
+---+--------+------+----------+----------+-----------+



In [30]:
from pyspark.sql.functions import lead
df.withColumn("lag_amount",lag("amount", 1).over(windowSpec)).show()

NameError: name 'lag' is not defined